In [ ]:
import torch 
from torch import nn 
from torch.utils.data import DataLoader 
from torchvision import datasets 
from torchvision.transforms import ToTensor

# 데이터 다운로드 

In [ ]:
training_data = datasets.FashionMNIST(
    root = 'data',
    train= True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor()
)


  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



# 데이터 로더 

In [ ]:
batch_size= 64

#데이터 로더 
train_dataloader = DataLoader(training_data, batch_size= batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for x,y in test_dataloader:
  print(x.shape,y.shape)
  break

torch.Size([64, 1, 28, 28]) torch.Size([64])


# 모델 만들기 
- 신경망 모델은 nn.Module을 상속받는 클래스를 생성하여 정의 
- __init__ 함수에서 레이어들을 정의 
- forward 함수에서 신경망 구조 설계 
    - forward함수에서 신경망에 데이터를 어떻게 전달할지 지정 
- 가능한 경우 GPU로 신경망을 이동시켜 연산을 가속 

In [89]:
device = 'cuda'

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
                                           nn.Linear(28*28,512),
                                           nn.ReLU(),
                                           nn.Linear(512,512),
                                           nn.ReLU(),
                                           nn.Linear(512,10)
                                           
                                          )
  def forward(self,x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits 

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# 모델 학습 
- 매개변수 최적화 
- 모델 학습을 위해서는 loss function 과 optimizer가 필요 


In [85]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)

In [86]:
#에폭 별 학습 모듈 
def train(dataloader, model, loss_fn, optimizer):
    size  = len(dataloader.dataset)
    #batch data 생성 
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

#한 에폭 끝날 때 마다 valid로 확인 
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [ ]:
epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303448  [    0/60000]
loss: 2.307097  [ 6400/60000]
loss: 2.312395  [12800/60000]
loss: 2.308623  [19200/60000]
loss: 2.313981  [25600/60000]
loss: 2.309192  [32000/60000]
loss: 2.311556  [38400/60000]
loss: 2.313121  [44800/60000]
loss: 2.305508  [51200/60000]
loss: 2.303329  [57600/60000]
Test Error: 
 Accuracy: 9.3%, Avg loss: 2.308857 

Epoch 2
-------------------------------
loss: 2.303448  [    0/60000]
loss: 2.307097  [ 6400/60000]
loss: 2.312395  [12800/60000]
loss: 2.308623  [19200/60000]
loss: 2.313981  [25600/60000]
loss: 2.309192  [32000/60000]
loss: 2.311556  [38400/60000]
loss: 2.313121  [44800/60000]
loss: 2.305508  [51200/60000]
loss: 2.303329  [57600/60000]
Test Error: 
 Accuracy: 9.3%, Avg loss: 2.308857 

Epoch 3
-------------------------------
loss: 2.303448  [    0/60000]
loss: 2.307097  [ 6400/60000]
loss: 2.312395  [12800/60000]
loss: 2.308623  [19200/60000]
loss: 2.313981  [25600/60000]
loss: 2.309192  [32000/60000

## 모델 저장 
- 모델을 저장하는 일반적인 방법은 내부 상태 사전을 직렬화 하는 것 

In [76]:
torch.save(model.state_dict(),'model.pth')

## 모델 불러오기 

In [79]:
model = NeuralNetwork()
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

# 추론 

In [80]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval() 
"""
eval 은 추론 모드로 모델을 변경시켜주는 것 
model.eval()를 해줌으로 써 layer 내부의 dropout, BatchNorm 과 같은 것을 꺼줌
no_grad()도 역시 추론 모드를 위해 하는 것으로 기울기 연산을 꺼줌  
"""
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


# Appendix : Auto Grad 
- 신경망을 학습할 때 역전파가 시행 됨 
- 매개변수는 역전파를 통해 손실 함수의 변화도에 따라 조정 됨 
- 이러한 변화도를 계산하기 위해 PyTorch에는 torch.autograd라고 불리는 자동 미분 엔진이 내장 
- 모든 계산 그래프에 대한 변화도의 자동 계산을 지원 

In [1]:
import torch
"""
입력 x, 매개변수 w와 b 그리고 일부 손실함수가 있는 간단한 신경망 가정 
x가 input, y가 output 
z = w*x + b 
z는 y값을 근사하는 것이 목적 
""" 
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output

w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

"""
여기서 w와 b는 최적화를 해야 하는 매개변수
매개변수를 정의할 때 requires_grad = True로 선언하면 됨 
x = torch.randn(5,3, requires_grad = True) or x.requires_grad(True)
"""

'\n여기서 w와 b는 최적화를 해야 하는 매개변수\n매개변수를 정의할 때 requires_grad = True로 선언하면 됨 \nx = torch.randn(5,3, requires_grad = True) or x.requires_grad(True)\n'

## 변화도 계산하기 

In [9]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0656, 0.1574, 0.1876],
        [0.0656, 0.1574, 0.1876],
        [0.0656, 0.1574, 0.1876],
        [0.0656, 0.1574, 0.1876],
        [0.0656, 0.1574, 0.1876]])
tensor([0.0656, 0.1574, 0.1876])


## 변화도 추적 멈추기 
- 학습이 종료된 뒤에는 grad를 계산할 필요가 없음 
- 이러한 경우 순전파만 사용되기 때문에 with torch.no_grad()를 통해 역전파를 끔 

In [3]:
with torch.no_grad():
  z = torch.matmul(x,w)+b 

tensor([1., 1., 1., 1., 1.])